In [1]:
import tensorflow as tf
import numpy as np
from keras.utils import to_categorical

D:\Python3.6\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(x_train.shape)
print(x_test.shape)

x_train = x_train[:1000]
x_test = x_test[:100]
y_train = y_train[:1000]
y_test = y_test[:100]

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

element_size = 28
time_steps = 28
num_classes = 10
batch_size = 100
hidden_layer_size = 128

_input = tf.placeholder(tf.float32, shape=[None, time_steps, element_size], name='inputs')
y = tf.placeholder(tf.float32, shape=[None, num_classes], name='label')

(60000, 28, 28)
(10000, 28, 28)


In [3]:
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
            tf.summary.scalar('stddev', stddev)
            tf.summary.scalar('max', tf.reduce_max(var))      
            tf.summary.scalar('min', tf.reduce_min(var))      
            tf.summary.histogram('histogram', var)

In [4]:
with tf.name_scope('rnn_weights'):
    with tf.name_scope('W_x'):
        Wx = tf.Variable(tf.zeros([element_size, hidden_layer_size]))
        variable_summaries(Wx)
    with tf.name_scope('W_h'):
        Wh = tf.Variable(tf.zeros([hidden_layer_size, hidden_layer_size]))
        variable_summaries(Wh)
    with tf.name_scope('Bias'):
        b_rnn = tf.Variable(tf.zeros([hidden_layer_size]))
        variable_summaries(b_rnn)

In [5]:
def rnn_step(previous_hidden_state, x):
    current_hidden_state = tf.tanh(tf.matmul(x, Wx) + tf.matmul(previous_hidden_state, Wh) + b_rnn)
    return current_hidden_state

In [6]:
processed_input = tf.transpose(_input, perm=(1,0,2))

initial_hidden = tf.zeros([batch_size, hidden_layer_size])

# tf.scan 有累加的效果
all_hidden_state = tf.scan(rnn_step,
                          processed_input,
                          initializer=initial_hidden,
                          name='states')

In [7]:
with tf.name_scope('linear_layer_weights') as scope:
    with tf.name_scope('W_linear'):
        Wl = tf.Variable(tf.truncated_normal([hidden_layer_size, num_classes], mean=0, stddev=0.01))
        variable_summaries(Wl)
    with tf.name_scope('Bias_linear'):
        bl = tf.Variable(tf.truncated_normal([num_classes], mean=0, stddev=0.01))
        variable_summaries(bl)

In [8]:
def get_linear_layer(hidden_state):
    return tf.matmul(hidden_state, Wl) + bl

In [9]:
with tf.name_scope('linear_layer_weights') as scope:
    all_outputs = tf.map_fn(get_linear_layer, all_hidden_state)
    output = all_outputs[-1]
    tf.summary.histogram('outputs', output)

In [10]:
with tf.name_scope('cross_entropy'):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=output))
    tf.summary.scalar('cross_entropy', cross_entropy)
    
with tf.name_scope('train'):
    gd = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cross_entropy)

with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(output, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))*100
    tf.summary.scalar('accuracy', accuracy)

merged = tf.summary.merge_all()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [14]:
STEPS = 100
MINIBATCH_SIZE = batch_size
LOG_DIR = 'F:/dlcv_experiment/tensorflow_tutorial/OReily_Tensorflow/'
init = tf.global_variables_initializer()

n_train = int(x_train.shape[0]/MINIBATCH_SIZE)
split_X = np.split(x_train, int(x_train.shape[0]/MINIBATCH_SIZE))
split_y = np.split(y_train, int(y_train.shape[0]/MINIBATCH_SIZE))

test_data = x_test[:MINIBATCH_SIZE].reshape((-1, time_steps, element_size)) 
test_label = y_test[:MINIBATCH_SIZE]

with tf.Session() as sess:
    train_writer = tf.summary.FileWriter(LOG_DIR+'/trian', graph=tf.get_default_graph())
    test_writer = tf.summary.FileWriter(LOG_DIR+'/test', graph=tf.get_default_graph())
    
    sess.run(init)
    
    for i in range(STEPS):
        for k in range(n_train):
            batch_x, batch_y = split_X[k], split_y[k]
            batch_x = batch_x.reshape([batch_size, time_steps, element_size])
            
            summary, _ = sess.run([merged, gd], feed_dict={_input:batch_x, y:batch_y})
            train_writer.add_summary(summary, i)
            
            if i % 10 == 0:
                acc, loss = sess.run([accuracy,cross_entropy], feed_dict={_input: batch_x, y: batch_y})
                print("Iter " + str(i) + ", Minibatch Loss= " + "{:.6f}".format(loss) + ", Training Accuracy= " + "{:.5f}".format(acc))
            
            if i % 20:
                summary, acc = sess.run([merged, accuracy], feed_dict={_input:test_data, y:test_label})
                train_writer.add_summary(summary, i)
            
    test_acc = sess.run(accuracy, feed_dict={_input:test_data, y:test_label})
    print("Test Accuracy:", test_acc)

Iter 0, Minibatch Loss= 2.303760, Training Accuracy= 6.00000
Iter 0, Minibatch Loss= 2.301728, Training Accuracy= 14.00000
Iter 0, Minibatch Loss= 2.303330, Training Accuracy= 8.00000
Iter 0, Minibatch Loss= 2.301440, Training Accuracy= 14.00000
Iter 0, Minibatch Loss= 2.302683, Training Accuracy= 10.00000
Iter 0, Minibatch Loss= 2.301846, Training Accuracy= 12.00000
Iter 0, Minibatch Loss= 2.302303, Training Accuracy= 12.00000
Iter 0, Minibatch Loss= 2.303531, Training Accuracy= 5.00000
Iter 0, Minibatch Loss= 2.301851, Training Accuracy= 12.00000
Iter 0, Minibatch Loss= 2.302931, Training Accuracy= 6.00000
Iter 10, Minibatch Loss= 2.301774, Training Accuracy= 10.00000
Iter 10, Minibatch Loss= 2.300106, Training Accuracy= 11.00000
Iter 10, Minibatch Loss= 2.302482, Training Accuracy= 8.00000
Iter 10, Minibatch Loss= 2.298291, Training Accuracy= 13.00000
Iter 10, Minibatch Loss= 2.301248, Training Accuracy= 10.00000
Iter 10, Minibatch Loss= 2.301404, Training Accuracy= 10.00000
Iter 10